## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)  # (50000, 32, 32, 3)

# Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7) 
    
    return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

In [0]:
def scheduler(epoch, lr):
    init_lr = 1e-3

    if epoch < 12:
        return init_lr
    elif epoch >= 12 and epoch < 15:
        return init_lr / 2
    elif epoch >= 15 and epoch < 20:
        return init_lr / 10

In [5]:
l2 = 3e-3
callbacks = [
    LearningRateScheduler(scheduler, verbose=1),
]

classifier = Sequential()

# 卷積組合
classifier.add(Conv2D(32, 3, padding='same', input_shape=(32, 32, 3),
                      activation = 'relu',
                      kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization())
classifier.add(Conv2D(32, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

# 卷積組合
classifier.add(Conv2D(64, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization())
classifier.add(Conv2D(64, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(l2)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

# FC
classifier.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2)))

classifier.add(Dropout(0.5))

# 輸出
classifier.add(Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(l2)))

# 超過兩個就要選 categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
classifier.fit(x_train, y_train, batch_size=128, epochs=18, validation_split=0.2, callbacks=callbacks)

Train on 40000 samples, validate on 10000 samples
Epoch 1/18

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
40000/40000 [==============================] - 9s 223us/step - loss: 2.9206 - acc: 0.4104 - val_loss: 2.7285 - val_acc: 0.3938
Epoch 2/18

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
40000/40000 [==============================] - 7s 167us/step - loss: 2.1534 - acc: 0.5728 - val_loss: 1.7962 - val_acc: 0.6633
Epoch 3/18

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
40000/40000 [==============================] - 7s 165us/step - loss: 1.8197 - acc: 0.6503 - val_loss: 1.7324 - val_acc: 0.6707
Epoch 4/18

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
40000/40000 [==============================] - 7s 168us/step - loss: 1.6453 - acc: 0.7020 - val_loss: 1.5899 - val_acc: 0.7135
Epoch 5/18

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
40000/40000 [==============================] - 

In [6]:
loss, acc = classifier.evaluate(x_test, y_test)
print('test loss: {:.4f}'.format(loss))
print('test acc: {:.4f}'.format(acc))

10000/10000 [==============================] - 1s 124us/step
test loss: 0.9473
test acc: 0.8232


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example = (np.zeros(shape=(1, 32, 32, 3)) - mean_train) / (std_train + 1e-7) 
classifier.predict(input_example)

array([[0.21921903, 0.00292347, 0.33102608, 0.08667612, 0.195629  ,
        0.02711849, 0.0896173 , 0.02668261, 0.01805988, 0.00304805]],
      dtype=float32)